In [129]:
def cross(a,b):
    return [s+t for s in a for t in b]

In [130]:
rows="ABCDEFGHI"
cols="123456789"
boxes=cross(rows,cols)
stringSudoku='..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

In [143]:
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [144]:
def display(values):
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [131]:
def dictSudoku(boxes,sud):
    sudoku={}
    for i in range(len(sud)):
        sudoku[boxes[i]]=sud[i]
    return sudoku

In [132]:
#modified sudoku by filling missing values
def boxValues(grid):
    sudoku={}
    for i in grid:
        if(grid[i]=='.'):
            sudoku[i]='123456789'
        else:
            sudoku[i]=grid[i]
    return sudoku

In [133]:
sudoku=dictSudoku(boxes,stringSudoku)
values=boxValues(sudoku)

In [134]:
display(sudoku)

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


In [135]:
display(values)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [138]:
def eliminate_o(values):
    def peers(A):
        if('ABC'.find(A[0])+1):
            first='ABC'
        elif('DEF'.find(A[0])+1):
            first='DEF'
        elif('GHI'.find(A[0])+1):
            first='GHI'
        if('123'.find(A[1])+1):
            second='123'
        elif('456'.find(A[1])+1):
            second='456'
        elif('789'.find(A[1])+1):
            second='789'
        return [s+t for s in first for t in second]

    for i in values:
        if(len(values[i])!=1):
            for c in cols:
                if(len(values[i[0]+c])==1 and (i[0]+c)!=i):
                    values[i]=values[i].replace(values[i[0]+c],'')
            for r in rows:
                if(len(values[r+i[1]])==1 and (r+i[1])!=i):
                    values[i]=values[i].replace(values[r+i[1]],'')
            for cr in peers(i):
                if(len(values[cr])==1 and cr!=i):
                    values[i]=values[i].replace(values[cr],'')
    return values          

In [139]:
def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values       

In [140]:
def only_choice(values):
    for box in unitlist:
        #print(box)
        for digit in '123456789':
            collect=[]
            for cell in box:
                if digit in values[cell]:
                    collect.append(cell)
            if(len(collect)==1):
                values[collect[0]]=digit
    return values

In [141]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        sudoku=eliminate(values)
        sudoku=only_choice(sudoku)

        # Your code here: Use the Only Choice Strategy

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [146]:
display(reduce_puzzle(values))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 
